In [3]:
import gzip
import shutil
import sqlite3
import pandas as pd
import random
from math import ceil

'''
# 日本語wordnetをDLして解凍
! wget "http://compling.hss.ntu.edu.sg/wnja/data/1.1/wnjpn.db.gz"  # 1~2分

with gzip.open('wnjpn.db.gz', 'rb') as f_in:
    with open('wnjpn.db', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# synset(概念ID)とlemma(単語)の組み合わせDataFrameの作成
conn = sqlite3.connect("wnjpn.db")
q = 'SELECT synset,lemma FROM sense,word USING (wordid) WHERE sense.lang="jpn"'
sense_word = pd.read_sql(q, conn)

# 類義語をリストにして返す関数
def get_synonyms(word):
    synsets = sense_word.loc[sense_word.lemma == word, "synset"]
    synset_words = set(sense_word.loc[sense_word.synset.isin(synsets), "lemma"])

    if word in synset_words:
        synset_words.remove(word)

    return list(synset_words)
'''

'\n# 日本語wordnetをDLして解凍\n! wget "http://compling.hss.ntu.edu.sg/wnja/data/1.1/wnjpn.db.gz"  # 1~2分\n\nwith gzip.open(\'wnjpn.db.gz\', \'rb\') as f_in:\n    with open(\'wnjpn.db\', \'wb\') as f_out:\n        shutil.copyfileobj(f_in, f_out)\n\n# synset(概念ID)とlemma(単語)の組み合わせDataFrameの作成\nconn = sqlite3.connect("wnjpn.db")\nq = \'SELECT synset,lemma FROM sense,word USING (wordid) WHERE sense.lang="jpn"\'\nsense_word = pd.read_sql(q, conn)\n\n# 類義語をリストにして返す関数\ndef get_synonyms(word):\n    synsets = sense_word.loc[sense_word.lemma == word, "synset"]\n    synset_words = set(sense_word.loc[sense_word.synset.isin(synsets), "lemma"])\n\n    if word in synset_words:\n        synset_words.remove(word)\n\n    return list(synset_words)\n'

In [4]:
#get_synonyms("データ")

In [5]:
! wget "http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt"
stop_words = pd.read_csv("Japanese.txt",header=None)[0].to_list()        

--2022-09-11 17:22:06--  http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt
Resolving svn.sourceforge.jp (svn.sourceforge.jp)... 44.240.209.230
Connecting to svn.sourceforge.jp (svn.sourceforge.jp)|44.240.209.230|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2202 (2.2K) [text/plain]
Saving to: ‘Japanese.txt.2’

Japanese.txt.2      100%[===================>]   2.15K  --.-KB/s    in 0s      

2022-09-11 17:22:06 (661 MB/s) - ‘Japanese.txt.2’ saved [2202/2202]



In [6]:
# MeCabをインストール
!pip install mecab-python3
!pip install ipadic
import MeCab
import ipadic

def wakati_text(text, hinshi=['名詞', '動詞']):
    m = MeCab.Tagger(ipadic.MECAB_ARGS)
    p = m.parse(text)
    p_split = [i.split("\t") for i in p.split("\n")][:-2]

    # 原文の分かち書き
    raw_words = [x[0] for x in p_split]

    # 同義語検索用の単語の原型リスト（品詞を絞る）
    second_half = [x[1].split(",") for x in p_split]
    original_words = [x[6] if x[0] in hinshi else "" for x in second_half]
    original_words = ["" if word in stop_words else word for word in original_words]

    return raw_words, original_words

In [7]:
text = "類似するデータを生成する記事を書いてます。"
raw_words, original_words= wakati_text(text)

print(raw_words)
# > ['類似', 'する', 'データ', 'を', '生成', 'する', '記事', 'を', '書い', 'て', 'ます', '。']
print(original_words)
# > ['類似', 'する', 'データ', '', '生成', 'する', '記事', '', '書く', '', '', '']

['類似', 'する', 'データ', 'を', '生成', 'する', '記事', 'を', '書い', 'て', 'ます', '。']
['類似', 'する', 'データ', '', '生成', 'する', '記事', '', '書く', '', '', '']


# Synonym replacement（ランダムに単語を同義語でn個置き換える)

In [62]:
import pandas as pd
# データの読み込み
df = pd.read_csv("all_text.tsv", 
                 delimiter='\t', header=None, names=['media_name', 'label','title','sentence'])

mn = df.media_name.values
labels = df.label.values
titles = df.title.values
sentences = df.sentence.values

In [83]:
from transformers import T5Tokenizer, RobertaForMaskedLM
import torch

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading
# masktoken = tokenizer.convert_tokens_to_ids('[MASK]')


def detokenize(ts): #s_replace you detokenize method
    sentence = ''
    for i in range(len(ts)):
        print(i, ts[i])
        if (ts[i] != '[CLS]'):
            if (ts[i] != '▁'):
                sentence += ts[i]
    return sentence

def synreplace(textlist):
    # textlist: honbun no list
    new_text = []
    model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")
    for i in range(len(textlist)):
        # text_orig = textlist[i]
        # prepend [CLS]
        text = "[CLS]" + textlist[i]
        # tokenize
 # hazusu      # tokens = tokenizer.tokenize(text)
        tokens = textlist[i]
        # chikan shiro
        masked_idx = random.randint(2, len(tokens)-1)
        print(masked_idx, len(tokens), tokens[masked_idx])
  #      tokens[n] = masktoken
        # masked_idx = tokens.index('[MASK]')
        # suironn
        tokens[masked_idx] = tokenizer.mask_token
        # convert to ids
  #      # token_ids = tokenizer.convert_tokens_to_ids(tokens)
        # convert to tensor
        # print(type(tokens))
        token_tensor = torch.tensor(tokens)
        # get the top 10 predictions of the masked token
        model = model.eval()
        with torch.no_grad():
            outputs = model(token_tensor)
            predictions = outputs[0][0, masked_idx].topk(1)
        for i, index_t in enumerate(predictions.indices):
            index = index_t.item()
            token = tokenizer.convert_ids_to_tokens([index])[0]
        # saichikan
        tokens[n] = token
        detokens = detokenize(tokens)
 # hazusu      # toketokens = tokenizer.tokenize(detokens)
        new_text.append(toketokens)
    return new_text

In [84]:
import numpy as np
raw_ids = np.array([[9, 3652, 26, 13508, 12, 19860, 57, 9123, 20, 7, 1393, 4221, 1429, 15133, 442, 4569, 20, 19133, 1272, 8, 5144, 2596, 612, 2855, 3244, 13426, 27, 8, 16058, 5448, 147, 318, 318, 774, 577, 1300, 559, 11, 4180, 12, 12079, 3413, 344, 229, 6741, 4294, 478, 20977, 10338, 7, 945, 7, 2416, 26, 27138, 12, 8988, 6671, 1068, 5176, 15710, 9146, 8, 2063], [9, 3652, 26, 13508, 12, 19860, 57, 9123, 20, 7, 1393, 4221, 1429, 15133, 442, 4569, 20, 19133, 1272, 8, 5144, 2596, 612, 2855, 3244, 13426, 27, 8, 16058, 5448, 147, 318, 318, 774, 577, 1300, 559, 11, 4180, 12, 12079, 3413, 344, 229, 6741, 4294, 478, 20977, 10338, 7, 945, 7, 2416, 26, 27138, 12, 8988, 6671, 1068, 5176, 15710, 9146, 8, 2063]])
synreplace(raw_ids)

# >データ  →  見聞
# >生成  →  創り出す
# >原文：類似のデータを生成する記事を書いてます。
# >変換後：類似の見聞を創り出すする記事を書いてます。

23 64 2855


ValueError: invalid literal for int() with base 10: '[MASK]'

9, 3652, 26, 13508, 12, 19860, 57, 9123, 20, 7, 1393, 4221, 1429, 15133, 442, 4569, 20, 19133, 1272, 8, 5144, 2596, 612, 2855, 3244, 13426, 27, 8, 16058, 5448, 147, 318, 318, 774, 577, 1300, 559, 11, 4180, 12, 12079, 3413, 344, 229, 6741, 4294, 478, 20977, 10338, 7, 945, 7, 2416, 26, 27138, 12, 8988, 6671, 1068, 5176, 15710, 9146, 8, 2063

# Random insertion（ランダムに単語をn個挿入）

In [7]:
'''
def random_insertion(raw_words, original_words, n):
    new_words = raw_words.copy()
    for _ in range(n):
        add_word(new_words, original_words)
    print(f"原文：{(''.join(raw_words))}")
    print(f"変換後：{(''.join(new_words))}")
    print("-"*50)
    return new_words

def add_word(new_words, original_words):
    synonyms = []
    counter = 0
    insert_word_original = [x for x in original_words if x]
    while len(synonyms) < 1:
        random_word = insert_word_original[random.randint(0, len(insert_word_original)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    print(f"挿入する単語：{random_synonym}")
    new_words.insert(random_idx, random_synonym)
'''

'\ndef random_insertion(raw_words, original_words, n):\n    new_words = raw_words.copy()\n    for _ in range(n):\n        add_word(new_words, original_words)\n    print(f"原文：{(\'\'.join(raw_words))}")\n    print(f"変換後：{(\'\'.join(new_words))}")\n    print("-"*50)\n    return new_words\n\ndef add_word(new_words, original_words):\n    synonyms = []\n    counter = 0\n    insert_word_original = [x for x in original_words if x]\n    while len(synonyms) < 1:\n        random_word = insert_word_original[random.randint(0, len(insert_word_original)-1)]\n        synonyms = get_synonyms(random_word)\n        counter += 1\n        if counter >= 10:\n            return\n    random_synonym = synonyms[0]\n    random_idx = random.randint(0, len(new_words)-1)\n    print(f"挿入する単語：{random_synonym}")\n    new_words.insert(random_idx, random_synonym)\n'

# Random deletion（ランダムに単語をn個削除する）

In [8]:
def random_deletion(words, p):
    # 1文字しかなければ削除しない
    if len(words) == 1:
        return words

    # 確率pでランダムに削除
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)
        else:
            print(f"削除：{word}")

    # 全て削除してしまったら、ランダムに1つ単語を返す
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    print(f"原文：{(''.join(words))}")
    print(f"変換後：{(''.join(new_words))}")
    print("-"*50)
    return new_words

# Random swap（ランダムに単語の場所をn回入れ替える）

In [9]:
def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        nwords = swap_word(new_words)

    print(f"原文：{(''.join(words))}")
    print(f"変換後：{(''.join(new_words))}")
    print("-"*50)
    return new_words

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    print(new_words[random_idx_1], "⇔", new_words[random_idx_2])

    return new_words

In [10]:
#synonym_replacement(raw_words, original_words, 2)

# >データ  →  見聞
# >生成  →  創り出す
# >原文：類似のデータを生成する記事を書いてます。
# >変換後：類似の見聞を創り出すする記事を書いてます。

In [11]:
#random_insertion(raw_words,original_words, 2)

# >挿入する単語：執筆
# >挿入する単語：類似性
# >原文：類似のデータを生成する記事を書いてます。
# >変換後：類似性類似するデータを生成する記事執筆を書いてます。

In [16]:
random_deletion(raw_words, 0.25)
# >削除：データ
# >削除：ます
# >原文：類似のデータを生成する記事を書いてます。
# >変換後：類似のを生成する記事を書いて。

削除：類似
削除：生成
原文：類似するデータを生成する記事を書いてます。
変換後：するデータをする記事を書いてます。
--------------------------------------------------


['する', 'データ', 'を', 'する', '記事', 'を', '書い', 'て', 'ます', '。']

In [13]:
random_swap(raw_words, 2)
# >の ⇔ を
# >を ⇔ 類似
# >原文：類似のデータを生成する記事を書いてます。
# >変換後：を類似データを生成する記事の書いてます。

記事 ⇔ を
記事 ⇔ データ
原文：類似するデータを生成する記事を書いてます。
変換後：類似する記事を生成するをデータ書いてます。
--------------------------------------------------


['類似', 'する', '記事', 'を', '生成', 'する', 'を', 'データ', '書い', 'て', 'ます', '。']

# 実行用関数

In [17]:
#def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
def eda(sentence, alpha_rs=0.1, p_rd=0.1, num_aug=2):
    
    # 分かち書き
    raw_words, original_words = wakati_text(sentence)
    num_words = len(raw_words)

    augmented_sentences = []
    techniques = ceil(alpha_rs) + ceil(p_rd)
    if techniques == 0:
        return

    num_new_per_technique = int(num_aug/techniques)+1

    '''
    #ランダムに単語を同義語でn個置き換える
    if (alpha_sr > 0):
        n_sr = max(1, int(alpha_sr*num_words))
        for _ in range(num_new_per_technique):
            a_words = synonym_replacement(raw_words,original_words ,n_sr)
            augmented_sentences.append(''.join(a_words))
    
    #ランダムに文中に出現する単語の同義語をn個挿入
    if (alpha_ri > 0):
        n_ri = max(1, int(alpha_ri*num_words))
        for _ in range(num_new_per_technique):
            a_words = random_insertion(raw_words,original_words, n_ri)
            augmented_sentences.append(''.join(a_words))
    '''

    #ランダムに単語の場所をn回入れ替える
    if (alpha_rs > 0):
        n_rs = max(1, int(alpha_rs*num_words))
        for _ in range(num_new_per_technique):
            a_words = random_swap(raw_words, n_rs)
            augmented_sentences.append(''.join(a_words))

    #ランダムに単語を確率pで削除する
    if (p_rd > 0):
        for _ in range(num_new_per_technique):
            a_words = random_deletion(raw_words, p_rd)
            augmented_sentences.append(''.join(a_words))

    #必要な文章の数だけランダムに抽出
    random.shuffle(augmented_sentences)
    augmented_sentences = augmented_sentences[:num_aug]

    #原文もリストに加える
    augmented_sentences.append(sentence)

    return augmented_sentences

In [19]:
# 原文から5つの文章を生成
#eda(text, alpha_sr=0.05, alpha_ri=0.05, alpha_rs=0.05, p_rd=0.05, num_aug=5)
eda(text, alpha_rs=0.05, p_rd=0.05, num_aug=5)


# >['類似するを生成する記事を書いてます。',
# '類似するデータを生成類似性する記事を書いてます。',
# '類似するデータを生成する記事を起草てます。',
# '類似するデータをますする記事を書いて生成。',
# '類似する書いを生成する記事をデータてます。',
# '類似するデータを生成する記事を書いてます。']

ます ⇔ する
原文：類似するデータを生成する記事を書いてます。
変換後：類似するデータを生成ます記事を書いてする。
--------------------------------------------------
記事 ⇔ を
原文：類似するデータを生成する記事を書いてます。
変換後：類似するデータを生成するを記事書いてます。
--------------------------------------------------
ます ⇔ する
原文：類似するデータを生成する記事を書いてます。
変換後：類似ますデータを生成する記事を書いてする。
--------------------------------------------------
削除：生成
削除：する
原文：類似するデータを生成する記事を書いてます。
変換後：類似するデータを記事を書いてます。
--------------------------------------------------
原文：類似するデータを生成する記事を書いてます。
変換後：類似するデータを生成する記事を書いてます。
--------------------------------------------------
削除：生成
原文：類似するデータを生成する記事を書いてます。
変換後：類似するデータをする記事を書いてます。
--------------------------------------------------


['類似ますデータを生成する記事を書いてする。',
 '類似するデータをする記事を書いてます。',
 '類似するデータを記事を書いてます。',
 '類似するデータを生成ます記事を書いてする。',
 '類似するデータを生成するを記事書いてます。',
 '類似するデータを生成する記事を書いてます。']

In [4]:
a=0
b=1
c=2

# b=2, c=1

a=b
a+=1
b=3
print(a, b)

2 3


In [4]:
import numpy as np

x = np.zeros(3)
y = [1,1,1]

x=y

y += 1

print(x)

TypeError: 'int' object is not iterable

In [5]:
import random
x, y = random.randint(0,100)

print(x, y)

TypeError: cannot unpack non-iterable int object